To run this notebook, you need to already have a FitnessCalculatorConfiguration in Nexus. Therefore, I recomment to do the configuration and extraction through python before running this notebook:
```
python pipeline.py --step=configure_nexus --emodel=EMODEL_NAME --iteration_tag=ITERATION_TAG --ttype=TTYPE
python pipeline.py --step=configure_model_from_gene --emodel=EMODEL_NAME --iteration_tag=ITERATION_TAG --ttype=TTYPE
python pipeline.py --step=extract --emodel=EMODEL_NAME --iteration_tag=ITERATION_TAG --ttype=TTYPE
```

## Loading the FitnessCalculatorConfiguration

In [1]:
species = "mouse"
brain_region = "SSCX"
data_access_point = "nexus"
nexus_project = "ncmv3"
nexus_organisation = "bbp"
nexus_endpoint = "prod"
forge_path = "./forge.yml"

emodel = "L5_TPC:A_cAC"
ttype = "L5 ET_8"
iteration_tag = "v0"

This next cell will ask for your Nexus token:

In [2]:
from bluepyemodel.access_point.nexus import NexusAccessPoint

access_point = NexusAccessPoint(
    emodel=emodel,
    species=species,
    brain_region=brain_region,
    project="ncmv3",
    organisation="bbp",
    endpoint="https://bbp.epfl.ch/nexus/v1",
    forge_path=forge_path,
    ttype=ttype,
    iteration_tag=iteration_tag
)

 ·······································································································································································································································································································································································································································································································································································································································································································································································································································································································································································································································

In [3]:
fitness_config = access_point.get_fitness_calculator_configuration()

print(fitness_config)

Fitness Calculator Configuration:

Protocols:
   {'name': 'IDRest_140', 'stimuli': [{'amp': 0.34121546350888576, 'delay': 250.0, 'duration': 1100.0, 'holding_current': -0.11401041566083829, 'thresh_perc': 142.08060963003956, 'totduration': 2328.125}], 'recordings': [{'type': 'CompRecording', 'location': 'soma', 'name': 'IDRest_140.soma.v', 'variable': 'v'}], 'validation': False}
   {'name': 'IDRest_180', 'stimuli': [{'amp': 0.43078548215426415, 'delay': 250.0, 'duration': 1100.0, 'holding_current': -0.13150815001648405, 'thresh_perc': 179.37716917877748, 'totduration': 2358.695652173913}], 'recordings': [{'type': 'CompRecording', 'location': 'soma', 'name': 'IDRest_180.soma.v', 'variable': 'v'}], 'validation': False}
   {'name': 'IDRest_260', 'stimuli': [{'amp': 0.6267264227033006, 'delay': 250.0, 'duration': 1100.0, 'holding_current': -0.11797499909996986, 'thresh_perc': 260.96611007378897, 'totduration': 2300.0}], 'recordings': [{'type': 'CompRecording', 'location': 'soma', 'name': '

## Adding EFeature or Protocol

In [4]:
from bluepyemodel.evaluation.efeature_configuration import EFeatureConfiguration
from bluepyemodel.evaluation.protocol_configuration import ProtocolConfiguration

In [5]:
protocol_name = "bAP"

stimuli = [{
    "amp": None,
    "delay": 700,
    "duration": 4,
    "holding_current": None,
    "thresh_perc": 1000,
    "totduration": 1000
}]

recordings = [
    {
        "type": "CompRecording",
        "location": "soma",
        "name": f"{protocol_name}.soma.v",
        "variable": "v"
    },
    {
        "type": "somadistanceapic",
        "name": f"{protocol_name}.dend1.v",
        "variable": "v",
        "somadistance": 137.74,
        "seclist_name": "apical"
    },
    {
        "type": "somadistanceapic",
        "name": f"{protocol_name}.dend2.v",
        "variable": "v",
        "somadistance": 266.20,
        "seclist_name": "apical"
    },
    {
        "type": "somadistanceapic",
        "name": f"{protocol_name}.ca_prox_apic.v",
        "variable": "cai",
        "somadistance": 33.11,
        "seclist_name": "apical"
    },
    {
        "type": "somadistanceapic",
        "name": f"{protocol_name}.ca_prox_basal.v",
        "variable": "cai",
        "somadistance": 33.11,
        "seclist_name": "basal"
    },
    {
        "type": "nrnseclistcomp",
        "name": f"{protocol_name}.ca_soma.v",
        "variable": "cai",
        "comp_x": 0.5,
        "sec_index": 0,
        "seclist_name": "soma"
    },
    {
        "type": "nrnseclistcomp",
        "name": f"{protocol_name}.ca_ais.v",
        "variable": "cai",
        "comp_x": 0.5,
        "sec_index": 0,
        "seclist_name": "axon"
    },
]

protocol_config = ProtocolConfiguration(
    name=protocol_name,
    stimuli=stimuli,
    recordings=recordings,
    validation=False
)

fitness_config.protocols.append(protocol_config)

In [6]:
efeature_def = EFeatureConfiguration(
    efel_feature_name="Spikecount",
    protocol_name=protocol_name,
    recording_name="soma.v",
    mean=1.0,
    std=0.1,
    efel_settings={"strict_stiminterval": False},
    threshold_efeature_std=None,
)

fitness_config.efeatures.append(efeature_def)

## Saving the FitnessCalculatorConfiguration

In [7]:
access_point.store_fitness_calculator_configuration(fitness_config)

<action> _deprecate_one
<succeeded> True
<action> _register_one
<succeeded> True


## Trying if the optimisation work:

```
python pipeline.py --step=test_optimize --emodel=EMODEL_NAME --iteration_tag=ITERATION_TAG --ttype=TTYPE
```